In [7]:
import numpy as np
from pyscf import gto, scf, dft

from pyscf.tools.dump_mat import dump_rec
from sys import stdout

from pop_scheme import PopulationScheme, MullikenPopulation
from rdft       import Constraints, rdft

mol = gto.Mole()
mol.verbose = 0
mol.atom = '''
He 0.0 0.0 -2.0
H  0.0 0.0 -2.929352
'''
# He  0.0 0.0  2.0
# He  0.0 0.0  4.0
mol.basis = 'sto-3g'
# mol.basis = 'cc-pvdz'
mol.spin   = 0
mol.charge = 1
mol.build()

mf           = scf.RHF(mol)
mf.conv_tol  = 1e-9
mf.verbose   = 4
mf.max_cycle = 100
mf.kernel()

pop = mf.mulliken_pop();

pop_method = MullikenPopulation(mf, [[0],[1]])
cons       = Constraints(pop_method, [[1.87240], [0.12760]])
mf1 = rdft(mf, cons, [[1.0], [1.0]])
pop = mf.mulliken_pop();



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/1c/6xd0c0090wn2kgrbj54n7qfr0000gn/T/tmpfpr9kioe
max_memory 4000 MB (current use 0 MB)
Set gradient conv threshold to 3.16228e-05
init E= -3.29632470437667
  HOMO = -1.05138686859439  LUMO = 0.159979774339434
cycle= 1 E= -2.85323026055022  delta_E= 0.443  |g|= 0.0491  |ddm|= 1.04
  HOMO = -1.53199658154336  LUMO = -0.265075556935008
cycle= 2 E= -2.85433685648377  delta_E= -0.00111  |g|= 0.00814  |ddm|= 0.067
  HOMO = -1.52391001856954  LUMO = -0.267529524439143
cycle= 3 E= -2.85436864404819  delta_E= -3.18e-05  |g|= 5.59e-05  |ddm|= 0.0135
  HOMO = -1.52389097093678  LUMO = -0.267550657824476
cycle= 4 E= -2.85436864530338  delta_E= -1.26e-09  |g

In [2]:
mf           = scf.UHF(mol)
mf.conv_tol  = 1e-9
mf.verbose   = 0
mf.max_cycle = 100
mf.kernel()

pop = mf.mulliken_pop();

pop_method = MullikenPopulation(mf, [[0],[1]], do_spin_pop=False)
pop  = pop_method.get_pop()
dm   = mf.make_rdm1()
fock = mf.get_fock()

cons = Constraints(pop_method, [[1.0], [1.0]])
pop_minus_nelec_required = cons.get_pop_minus_nelec_required()
fock_add = cons.make_fock_add(dm, np.asarray([[1.0], [1.0]]), weight_matrices=None, do_spin_pop=True)

fock += fock_add

print("fock_add[0] = \n", fock_add[0])
print("fock_add[1] = \n", fock_add[1])

pop_method = MullikenPopulation(mf, [[0],[1]], do_spin_pop=True)
pop  = pop_method.get_pop()
dm   = mf.make_rdm1()
fock = mf.get_fock()


cons = Constraints(pop_method, [[0.5,0.5], [0.5,0.5]])
pop_minus_nelec_required = cons.get_pop_minus_nelec_required()
fock_add = cons.make_fock_add(dm, np.asarray([[1.0,1.0], [1.0,1.0]]), weight_matrices=None, do_spin_pop=True)

fock += fock_add

print("fock_add[0] = \n", fock_add[0])
print("fock_add[1] = \n", fock_add[1])

 ** Mulliken pop       alpha | beta **
pop of  0 He 1s        0.88620 | 0.88620   
pop of  1 H 1s        0.11380 | 0.11380   
In total             1.00000 | 1.00000   
 ** Mulliken atomic charges   ( Nelec_alpha | Nelec_beta ) **
charge of  0He =      0.22760  (     0.88620      0.88620 )
charge of  1H =      0.77240  (     0.11380      0.11380 )
fock_add[0] = 
 [[-0.11380066 -0.21720168]
 [-0.21720168 -0.88619934]]
fock_add[1] = 
 [[-0.11379972 -0.21720168]
 [-0.21720168 -0.88620028]]
fock_add[0] = 
 [[ 3.86199337e-01  1.24900090e-16]
 [ 1.24900090e-16 -3.86199337e-01]]
fock_add[1] = 
 [[ 3.86200280e-01  8.32667268e-17]
 [ 8.32667268e-17 -3.86200280e-01]]


In [3]:
-5.69010966e-02*2

-0.1138021932